In [1]:
import os

os.chdir('../')

%pwd

'c:\\Users\\P52s\\Documents\\Python Project\\SpellX'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    data_path : Path
    data_file : Path
    model_path : Path

In [3]:
from spellX.constants import *
from spellX.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig: 
        config = self.config.model_evaluation

        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            data_file = config.data_file,
            model_path = config.model_path
        )

        return model_evaluation_config

In [5]:
import pickle
from spellX.utils.evaluation import rouge_1_metrics, bleu_score
import yaml 
import datetime

class ModelEvaluator:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        self.avg_rouge_1 = (0, 0, 0)  # Initialize with zeros for precision, recall, and F1
        self.avg_bleu_1 = 0
        self.model_count = 0
        

    def get_model_filename(self):
        # Generate the filename for the current model
        model_path = f"{self.config.model_path}/model{self.model_count}.pkl"
        with open(model_path, 'rb') as file:
            sp = pickle.load(file)
        return sp

    def evaluate(self):
        # Load the pre-trained model
        model = self.get_model_filename()

        # Read the test data from spell-testset1.txt
        with open("./artifacts/data_ingestion/data/test/spell-testset1.txt", "r") as f:
            test_data = f.readlines()

        # Initialize lists to store ROUGE and BLEU scores
        rouge_1_scores = []
        bleu_1_scores = []

        # Correct sentences and calculate metrics
        for sentence in test_data:
            # Assuming your model takes a sentence and returns a corrected sentence
            corrected_sentence = model.spell_correct(sentence.strip())['spell_corrected_text']

            # Replace this with the corresponding reference sentences if you have them
            reference_sentence = sentence.strip()

            # Calculate ROUGE-1 metrics
            rouge_1_precision, rouge_1_recall, rouge_1_f1 = rouge_1_metrics(corrected_sentence, reference_sentence)

            # Calculate BLEU-1 score
            bleu_1 = bleu_score(corrected_sentence, reference_sentence, n=1)

            # Append scores to lists
            rouge_1_scores.append((rouge_1_precision, rouge_1_recall, rouge_1_f1))
            bleu_1_scores.append(bleu_1)

        # Calculate average ROUGE-1 and BLEU-1 scores
        total_rouge_1_precision = sum(precision for precision, _, _ in rouge_1_scores)
        total_rouge_1_recall = sum(recall for _, recall, _ in rouge_1_scores)
        total_rouge_1_f1 = sum(f1 for _, _, f1 in rouge_1_scores)
        total_bleu_1 = sum(bleu for bleu in bleu_1_scores)

        num_samples = len(test_data)

        self.avg_rouge_1 = (
            total_rouge_1_precision / num_samples,
            total_rouge_1_recall / num_samples,
            total_rouge_1_f1 / num_samples,
        )
        self.avg_bleu_1 = total_bleu_1 / num_samples

        return  self.avg_rouge_1 ,self.avg_bleu_1 

    def print_results(self):
        result = {
            "Average ROUGE-1 Precision": self.avg_rouge_1[0],
            "Average ROUGE-1 Recall": self.avg_rouge_1[1],
            "Average ROUGE-1 F1 Score": self.avg_rouge_1[2],
            "Average BLEU-1 Score": self.avg_bleu_1
        }

        # Print the results to the console
        for key, value in result.items():
            print(f"{key}: {value}")

        # Generate a unique filename with a timestamp
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        filename = f"{self.config.root_dir}/metrics_{timestamp}.yaml"

        # Save the results to the unique YAML file
        with open(filename, "w") as yaml_file:
            yaml.dump(result, yaml_file)

In [6]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluator(config=model_evaluation_config)
    model_evaluation_config.evaluate()
    model_evaluation_config.print_results()
except Exception as e:
    raise e

[2023-09-08 13:40:23,106: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-08 13:40:23,114: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-08 13:40:23,114: INFO: common: created directory at: artifacts]
[2023-09-08 13:40:23,122: INFO: common: created directory at: artifacts/model_evaluation]


[2023-09-08 13:40:33,622: INFO: symspell: number of possible corrections: 1]
[2023-09-08 13:40:33,622: INFO: symspell: edit distance for deletions: 3]
[2023-09-08 13:40:33,622: DEBUG: model: Symspell suggestions: [('contented', 1), ('contented', 1), ('contented', 1)]]
[2023-09-08 13:40:33,622: DEBUG: model: Phoneme suggestions: []]
[2023-09-08 13:40:33,638: INFO: symspell: number of possible corrections: 3]
[2023-09-08 13:40:33,638: INFO: symspell: edit distance for deletions: 3]
[2023-09-08 13:40:33,638: DEBUG: model: Symspell suggestions: [('contended', 1), ('contend', 1), ('contente', 1), ('contended', 1), ('contend', 1), ('contente', 1), ('contente', 1)]]
[2023-09-08 13:40:33,638: DEBUG: model: Phoneme suggestions: [('contente', 1), ('contente', 1), ('contente', 1), ('contend', 1), ('contend', 1), ('content', 2), ('content', 2), ('contained', 3), ('contained', 3), ('condemned', 3), ('condemned', 3), ('continued', 3), ('continued', 3)]]
[2023-09-08 13:40:33,645: INFO: symspell: numb